In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp '/content/drive/MyDrive/Colab Notebooks/corrected_ratings_corpus.txt' ./

In [6]:
from gensim import corpora

In [7]:
# 토큰의 순서를 고려하지 않고, 중복을 제거한 형태(set)로 LDA 피처를 만든다.
corpus_fname = './corrected_ratings_corpus.txt'
documents, tokenized_corpus = [], []
tokenizer = get_tokenizer('mecab')
with open(corpus_fname, 'r', encoding='utf-8') as f:
    for document in f:
        tokens = list(set(tokenizer.morphs(document.strip())))
        documents.append(document)
        tokenized_corpus.append(tokens)
dictionary = corpora.Dictionary(tokenized_corpus)
corpus = [dictionary.doc2bow(text) for text in tokenized_corpus]

In [8]:
from gensim.models import ldamulticore

### LdaMulticore  
* num_topics 토픽수 (K)


### get_document_topics : 각 문서별 토픽 분포를 리턴
* minimum_probability : 0.5 , 0.5 미만의 토픽 분포는 무시  
결과적으로 해당 문서에서 가장 높은 확률값을 지닌 토픽만 추출한다. 

In [9]:
LDA = ldamulticore.LdaMulticore(corpus, id2word=dictionary,
                                    num_topics=30,
                                    minimum_probability=0.0,
                                    workers=4)
# 특정 토픽의 확률이 0.5보다 클 경우에만 데이터를 리턴한다
# 확률의 합은 1이기 때문에 해당 토픽이 해당 문서에서 확률값이 가장 큰 토픽이 된다
all_topics = LDA.get_document_topics(corpus, minimum_probability=0.5, per_word_topics=False)

In [10]:
for doc_idx, topic in enumerate(all_topics[:5]):
  print(doc_idx, topic)

0 [(0, 0.5701347)]
1 [(23, 0.981761)]
2 []
3 [(2, 0.5212252)]
4 [(17, 0.93095237)]


In [11]:
!mkdir lda
output_fname = './lda/lda'

In [12]:
with open(output_fname + ".results", 'w') as f:
        for doc_idx, topic in enumerate(all_topics):
            if len(topic) == 1:
                topic_id, prob = topic[0]
                f.writelines(documents[doc_idx].strip() + "\u241E" + ' '.join(tokenized_corpus[doc_idx]) + "\u241E" + str(topic_id) + "\u241E" + str(prob) + "\n")
LDA.save(output_fname + ".model")

In [13]:
!cp -rf lda '/content/drive/MyDrive/Colab Notebooks'

# 사전작업 (토크나이저 준비)

In [ ]:
# 은전한닢 설치
!sudo apt-get install curl git
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)


!pip install konlpy
import sys, re, argparse
from konlpy.tag import Okt, Komoran, Mecab, Hannanum, Kkma
def get_tokenizer(tokenizer_name):
    if tokenizer_name == "komoran": # 코모란
        tokenizer = Komoran()
    elif tokenizer_name == "okt": # Okt
        tokenizer = Okt()
    elif tokenizer_name == "mecab": # 은전한닢
        tokenizer = Mecab()
    elif tokenizer_name == "hannanum": # 한나눔
        tokenizer = Hannanum()
    elif tokenizer_name == "kkma":
        tokenizer = Kkma()
    elif tokenizer_name == "khaiii": # Khaiii
        tokenizer = KhaiiiApi()
    else:
        tokenizer = Mecab()
    return tokenizer

# 평가